<a href="https://colab.research.google.com/github/mathemusician/google_colab/blob/main/sparse_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/neuralmagic/sparseml
!cd sparseml/integrations/huggingface-transformers

Cloning into 'sparseml'...
remote: Enumerating objects: 293590, done.
remote: Counting objects: 100% (10435/10435), done.
remote: Compressing objects: 100% (3235/3235), done.
remote: Total 293590 (delta 7029), reused 10284 (delta 6924), pack-reused 283155
Receiving objects: 100% (293590/293590), 211.02 MiB | 23.24 MiB/s, done.
Resolving deltas: 100% (241953/241953), done.


In [ ]:
# copied from https://colab.research.google.com/drive/1_xkARB35307P0-BTnqMy0flmYrfoYi5T#scrollTo=igwruhYxE_a7
!pip uninstall -y torch torchvision torchaudio
# May need to change in the future if Colab no longer uses CUDA 11.0
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Found existing installation: torch 1.9.0+cu102
Uninstalling torch-1.9.0+cu102:
  Successfully uninstalled torch-1.9.0+cu102
Found existing installation: torchvision 0.10.0+cu102
Uninstalling torchvision-0.10.0+cu102:
  Successfully uninstalled torchvision-0.10.0+cu102
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1 MB 1.4 MB/s eta 0:03:43tcmalloc: large alloc 1147494400 bytes == 0x5565709aa000 @  0x7fb37233e615 0x5565379d602c 0x556537ab617a 0x5565379d8e4d 0x556537acac0d 0x556537a4d0d8 0x556537a47c35 0x5565379da73a 0x556537a4cf40 0x556537a47c35 0x5565379da73a 0x556537a4993b 0x556537acba56 0x556537a48fb3 0x556537acba56 0x556537a48fb3 0x556537acba56 0x556537a48fb3 0x5565379dab99 0x556537a1de79 0x5565379d97b2 0x556537a4ce65 0x556537a47c35 0x5565379da73a 0x556537a4993b 0x556537a47c35 0x5565379da73a 0x556537a48b0e 0x5565379da65a 0x556537a48d67 0x556537a47c35
     |█████████████████████████████▏  | 1055.7 MB 1.5 MB/s eta 0:01:

In [ ]:
!git clone https://github.com/neuralmagic/transformers.git
!cd /content/transformers; pip install -e .
!pip install datasets
!pip install sparseml[torch]
!pip install wandb

Cloning into 'transformers'...
remote: Enumerating objects: 73310, done.
remote: Total 73310 (delta 0), reused 0 (delta 0), pack-reused 73310
Receiving objects: 100% (73310/73310), 56.21 MiB | 22.20 MiB/s, done.
Resolving deltas: 100% (52175/52175), done.
Obtaining file:///content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3 MB 8.6 MB/s 
     |████████████████████████████████| 895 kB 39.1 MB/s 
  Running setup.py develop for transformers
     |████████████████████████████████| 264 kB 5.3 MB/s 
     |████████████████████████████████| 243 kB 34.2 MB/s 
     |████████████████████████████████| 119 kB 44.6 MB/s 
     |████████████████████████████████| 533 kB 5.3 MB/s 
     |████████████████████████████████| 7.4 MB 17.7 MB/s 
     |████████████████████████████████| 4.5 MB 40.3 MB/s 
     |████████████████████████████████| 80 kB 8.6 MB/s 
     |████████████

# Recipe

I've changed the default recipe slightly so that the run would finish faster:
 - num_train_epochs: 1; default: 30
 - preprocessing_num_workers: 1; default: 6
 - pad_to_max_length: True; default: not included
 - max_train_samples 20; default: not included
 - max_eval_samples 20; default: not included
 - max_predict_samples 20; default: not included

Obviously, this won't give you an impressive F1 score, but it does save you from waiting 173 hours. (944x speedup)

In [9]:
!cd sparseml/integrations/huggingface-transformers
!python transformers/examples/pytorch/question-answering/run_qa.py \
  --pad_to_max_length True \
  --model_name_or_path bert-base-uncased \
  --dataset_name squad \
  --do_train \
  --do_eval \
  --max_train_samples 20 \
  --max_eval_samples 20 \
  --max_predict_samples 20 \
  --evaluation_strategy epoch \
  --per_device_train_batch_size 16 \
  --learning_rate 5e-5 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir MODELS_DIR/bert-base-12layers_prune80 \
  --cache_dir cache \
  --preprocessing_num_workers 1 \
  --fp16 \
  --num_train_epochs 1 \
  --recipe sparseml/integrations/huggingface-transformers/recipes/bert-base-12layers_prune80.md \
  --onnx_export_path MODELS_DIR/bert-base-12layers_prune80/onnx \
  --save_strategy epoch \
  --save_total_limit 2 \
  --report_to wandb

2021-09-10 08:34:03 __main__     WARNING  Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
09/10/2021 08:34:03 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: True
2021-09-10 08:34:03 __main__     INFO     Training/evaluation parameters TrainingArguments(output_dir=MODELS_DIR/bert-base-12layers_prune80, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=IntervalStrategy.EPOCH, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/Sep10_08-34-03_20cc82d127dc, logging_strategy=IntervalStrategy.STEPS, loggi